In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip -q '/content/drive/My Drive/Colab Notebooks/train.zip' -d dataset
! unzip -q '/content/drive/My Drive/Colab Notebooks/test.zip' -d dataset

In [ ]:
# %tensorflow_version 1.x
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing import image_dataset
from keras.models import load_model
from sklearn.metrics import accuracy_score

In [ ]:
def load_data3(path, batch_size, split=False):
    if split:
      datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
      return datagen.flow_from_directory(
              path,
              target_size=(197, 197),
              shuffle = True,
              class_mode='categorical',
              batch_size=batch_size,
              subset='training'), datagen.flow_from_directory(
              path,
              target_size=(197, 197),
              class_mode='categorical',
              batch_size=batch_size,
              subset='validation')
    else:
      datagen = ImageDataGenerator(rescale=1./255)
      return datagen.flow_from_directory(
              path,
              target_size=(197, 197),
              shuffle=False,
              class_mode='categorical',
              batch_size=batch_size,
              color_mode='rgb'), None
def load_ds(train_path, test_path, batch_size, image_size):
    train_data = image_dataset.image_dataset_from_directory(
            train_path, 
            label_mode='categorical',
            batch_size=batch_size,
            image_size=(image_size, image_size), 
            shuffle=True, 
            seed=1337,
            validation_split=0.2,
            subset="training")
    val_data = image_dataset.image_dataset_from_directory(
            train_path, 
            label_mode='categorical',
            batch_size=batch_size,
            image_size=(image_size, image_size), 
            shuffle=True, 
            seed=1337,
            validation_split=0.2,
            subset="validation")
    test_data = image_dataset.image_dataset_from_directory(
            test_path, 
            label_mode='categorical',
            batch_size=batch_size,
            image_size=(image_size, image_size), 
            shuffle=False)
    return train_data, val_data, test_data

In [ ]:
# train_generator, _  = load_data('/content/train')
# test_generator, _  = load_data('/content/test')
test3_generator128, _ = load_data3('/content/test', 128)
test3_generator64, _ = load_data3('/content/test', 64)
_, _, test_ds197 = load_ds("/content/train", "/content/test", 128, 197)
_, _, test_ds48 = load_ds("/content/train", "/content/test", 128, 48)

Found 5512 images belonging to 7 classes.
Found 5512 images belonging to 7 classes.
Found 22035 files belonging to 7 classes.
Using 17628 files for training.
Found 22035 files belonging to 7 classes.
Using 4407 files for validation.
Found 5512 files belonging to 7 classes.
Found 22035 files belonging to 7 classes.
Using 17628 files for training.
Found 22035 files belonging to 7 classes.
Using 4407 files for validation.
Found 5512 files belonging to 7 classes.


In [ ]:
import numpy as np
# make an ensemble prediction for multi-class classification
def ensemble_predictions(models_TL):
  # make predictions
  yhats = np.zeros((len(models_TL),5512,7))

  for model_ind in range(len(models_TL)):
    (model, data) = models_TL[model_ind]
    yhat = model.predict(data)
    # print(yhat)
    # print(len(yhat))
    yhats[model_ind,:,:] = yhat

  summed = np.sum(yhats, axis=0)
  result = np.argmax(summed, axis=1)
  return result

In [ ]:
vgg_model = load_model('/content/drive/MyDrive/model/VGG16/VGG16_sgd_-EPOCHS_120-DROPOUT_0.5-test_acc_0.683.h5')
print('\n# Evaluate on test data')
results_test = vgg_model.evaluate(test_ds197)
print('test loss, test acc:', results_test)


# Evaluate on test data
44/44 [==============================] - 14s 316ms/step - loss: 0.9335 - accuracy: 0.6831
test loss, test acc: [0.9334798455238342, 0.6830551624298096]


In [ ]:
resnet50_model = load_model('/content/drive/MyDrive/model/Resnet50/resnet50-weights-best.h5')
print('\n# Evaluate on test data')
results_test = resnet50_model.evaluate(test3_generator128)
print('test loss, test acc:', results_test)


# Evaluate on test data
44/44 [==============================] - 7s 162ms/step - loss: 1.0349 - accuracy: 0.6856
test loss, test acc: [1.034889578819275, 0.6855950355529785]


In [ ]:
resnet18_model = load_model('/content/drive/MyDrive/model/RESNET18.h5')

In [ ]:
base_model = load_model('/content/drive/MyDrive/model/Baseline/baseline-EPOCHS_150-DROPOUT_0.3-test_acc_0.675.h5')
print('\n# Evaluate on test data')
results_test = base_model.evaluate(test_ds48)
print('test loss, test acc:', results_test)


# Evaluate on test data
44/44 [==============================] - 3s 79ms/step - loss: 1.5137 - accuracy: 0.6751
test loss, test acc: [1.5137064456939697, 0.6750725507736206]


In [ ]:
senet50_model = load_model('/content/drive/MyDrive/model/Senet50/senet50-weights-best.h5')
print('\n# Evaluate on test data')
results_test = senet50_model.evaluate(test3_generator64)
print('test loss, test acc:', results_test)


# Evaluate on test data
87/87 [==============================] - 9s 100ms/step - loss: 1.1435 - accuracy: 0.6239
test loss, test acc: [1.143450140953064, 0.623911440372467]


In [ ]:
models_TL = [(vgg_model, test_ds197), (resnet50_model, test3_generator128), (senet50_model, test3_generator64), (base_model, test_ds48)]

In [ ]:
result = ensemble_predictions(models_TL)
labels = test3_generator128.classes

In [ ]:
accuracy_score(result, labels)

0.7224238026124818